In [1]:
import pandas as pd
from prog.tools import *
from prog.db import *
import os, sqlalchemy

In [2]:
# 取得config
root = os.getcwd()
config_path = os.path.join(root, "prog", "config.json")
with open(config_path, 'r') as f:
    config = json.load(f)

In [3]:
db_con = config["database"]
db = Database(db_con)

df = db.get_data(db_con["table"]["predict"])#, start_time = "2024-06-13", end_time = "2024-06-19")
# df = df.sort_values("serial").reset_index(drop = True)
remove = (df[["length", "width"]] <= 0).any(axis = 1) # 長寬為0無法切割
df = df[~remove].reset_index(drop = True)
df = df[["order_id", "cabinet", "item_name", "color", "length", "width", "e_ship_date"]]
print(f"shape = {df.shape}")
print(f"remove: {remove.sum()}")
df.head(3)

shape = (1695, 7)
remove: 0


,order_id,cabinet,item_name,color,length,width,e_ship_date
0,BS11206028,00_共用,T腳,AL3612A_18,2500.0,120.0,2024-06-14
1,BS11206028,00_共用,T腳,AL3612A_18,2500.0,120.0,2024-06-14
2,BS11206028,00_共用,T腳,BK1185A_18,2500.0,120.0,2024-06-14


### Test cost time

In [4]:
def spend_time(mean_cut_time, df, cols):
    df1 = df.sort_values(cols).reset_index(drop = True)

    df_time = pd.DataFrame()
    g = df1.groupby("order_id")
    for group in g.size().index:
        df1_1 = g.get_group(group)
        if len(df1_1) > 1:
            df1_1 = df1_1.iloc[[1, -1]]
            df_time.loc[group, "start"] = df1_1.index[0]
            df_time.loc[group, "end"] = df1_1.index[1]
        
    df_time["gap"] = df_time["end"] - df_time["start"] + 1
    df_time["time"] = df_time["gap"] * mean_cut_time
    
    return df_time

In [5]:
mean_cut_time = ((8*60*60) / 8000) * 2
df_time1 = spend_time(mean_cut_time, df, cols = ["color", "length", "width"])
df_time2 = spend_time(mean_cut_time, df, cols = ["color", "order_id", "cabinet", "length", "width"])
df_time3 = spend_time(mean_cut_time, df, cols = ["order_id", "color", "cabinet", "length", "width"])

In [6]:
df_time = pd.concat([
        df_time1[["time"]].describe(),
        df_time2[["time"]].describe(),
        df_time3[["time"]].describe()
    ], axis = 1).round(2)
df_time.columns = ["color", "color/order_id", "order_id/color"]
df_time

,color,color/order_id,order_id/color
count,8.00,8.00,8.00
mean,7767.90,7711.20,1518.30
std,4957.65,4920.19,1300.65
min,21.60,21.60,21.60
25%,6541.20,6217.20,412.20
50%,8931.60,9100.80,1443.60
75%,11579.40,11453.40,2561.40
max,12067.20,11952.00,3290.40


In [7]:
df_gap = pd.concat([
        df_time1[["gap"]].describe(),
        df_time2[["gap"]].describe(),
        df_time3[["gap"]].describe()
    ], axis = 1).round(2)
df_gap.columns = ["color", "color/order_id", "order_id/color"]
df_gap

,color,color/order_id,order_id/color
count,8.00,8.00,8.00
mean,1078.88,1071.00,210.88
std,688.56,683.36,180.65
min,3.00,3.00,3.00
25%,908.50,863.50,57.25
50%,1240.50,1264.00,200.50
75%,1608.25,1590.75,355.75
max,1676.00,1660.00,457.00


### Sort

In [8]:
df = df.sort_values(["color", "order_id", "cabinet", "length", "width"], ascending = [True, True, True, False, False])
df = df.reset_index(drop = True)
df.head()

,order_id,cabinet,item_name,color,length,width,e_ship_date
0,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14
1,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14
2,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14
3,BS11206028,02_鞋櫃,背拉,*不拘_18,274.0,96.0,2024-06-14
4,BS11206028,02_鞋櫃,背拉,*不拘_18,274.0,96.0,2024-06-14


### Calculate area

In [9]:
df["area"] = df["length"] * df["width"] # 每個矩形的面積
length_limit = config["limit"]["length"]
width_limit = config["limit"]["width"]
area_limit = length_limit * width_limit
df["area_prob"] = df["area"] / area_limit # 矩形站箱子的面積
df.head(3)

,order_id,cabinet,item_name,color,length,width,e_ship_date,area,area_prob
0,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14,45984.0,0.005109
1,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14,45984.0,0.005109
2,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14,45984.0,0.005109


### Calculate waste and plate_id

In [10]:
df = calculate_waste(df, length_limit, width_limit)
df.head(3)

,order_id,cabinet,item_name,color,length,width,e_ship_date,area,area_prob,waste,plate_id
0,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14,45984.0,0.005109,0.994891,1
1,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14,45984.0,0.005109,0.989781,1
2,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14,45984.0,0.005109,0.984672,1


### 基礎耗損率

In [11]:
waste_score = calculate_mean_wast(df, waste_json = None)
waste_score["plate"]

{'waste': 0.1659, 'plate': 41, 'no_rm_waste': 0.4291, 'no_rm_plate': 88}

In [12]:
df.head(3)

,order_id,cabinet,item_name,color,length,width,e_ship_date,area,area_prob,waste,plate_id
0,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14,45984.0,0.005109,0.994891,1
1,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14,45984.0,0.005109,0.989781,1
2,BS11206028,01_鞋櫃,背拉,*不拘_18,479.0,96.0,2024-06-14,45984.0,0.005109,0.984672,1


In [13]:
df.query("(waste == 1) or (plate_id == 0)")

,order_id,cabinet,item_name,color,length,width,e_ship_date,area,area_prob,waste,plate_id
